In [2]:
import pickle
from glob import glob
import numpy as np

In [9]:
# ----------------------------------------------------------------------------------------------------------
# In this cell, we load all matching .pkl data files for 1-push tasks, extract episodes where a solution 
# was found, and compute the per-region success rates (solutions/pushes). We aggregate these results for 
# later analysis, making sure to not double-count regions within an episode.
# ----------------------------------------------------------------------------------------------------------

# Initialize counter for number of files processed
search_push_data = []
search_total_pushes = []

search_per_env_per_neighbor = {}
search_region_opening_attempts = 0
search_success_region_opening_attempts = 0
sols = 0
double_sols = 0
# Path to the data files (pkl files containing pickled episode results)
data_dir = "/common/users/dm1487/namo_data/nov30/aug9_envs/1_push_train/**/*.pkl"


xml_file_set = set()

# Iterate through all .pkl files that match the data_dir pattern
# BUG: glob() here refers to the glob function from the glob module, but glob.glob() 
# does not expand ** (recursive glob) unless you pass recursive=True.
for file in glob(data_dir, recursive=True):
    try:
        # Attempt to open and load each pickle file
        with open(file, "rb") as f:
            data = pickle.load(f)
            episode_results = data['episode_results']
            for ep_idx in range(len(episode_results)):
                xml_file_name = "_".join(episode_results[ep_idx]['xml_file'].split('/')[-4:])
                if "easy" in xml_file_name:
                    continue
                if xml_file_name not in xml_file_set:
                    xml_file_set.add(xml_file_name)
                    search_per_env_per_neighbor[xml_file_name] = {}
                
                if episode_results[ep_idx]['solution_found']:
                    sols += 1
                    region_label = episode_results[ep_idx]['algorithm_stats']['neighbour_region_label']
                    if region_label not in search_per_env_per_neighbor[xml_file_name]:
                        search_per_env_per_neighbor[xml_file_name][region_label] = []
                    search_per_env_per_neighbor[xml_file_name][region_label].append(episode_results[ep_idx]['max_depth_reached'])
                    if episode_results[ep_idx]['max_depth_reached'] > 1:
                        double_sols += 1
                        # print(episode_results[ep_idx])
            
    except Exception as e:
        # Print the error if something goes wrong, then continue to the next file
        print("Error in search data collection:")
        print(e)
        continue
    
print(sols, double_sols, double_sols/sols)


34493 0 0.0


In [10]:
count = 0
total_count = 0
for env in search_per_env_per_neighbor:
    for region in search_per_env_per_neighbor[env]:
        total_count += 1        
        if np.unique(np.array(search_per_env_per_neighbor[env][region]))[0] == 2:
            count += 1

print(count, total_count, count/total_count)


0 34493 0.0
